# Working with Symbolic Expressions

## 0.1 Finding an exact derivative with a computer algebra system

## 0.2 Doing symbolic algebra in Python

In [1]:
from math import sin
def f(x):
    return (3*x**2 + x) * sin(x)

# 1 Modeling algebraic expressions

## 1.1 Breaking an expression into pieces

## 1.2 Building an expression tree

## 1.3 Translating the expression tree to Python

In [2]:
class Power():
    def __init__(self,base,exponent):
        self.base = base
        self.exponent = exponent

In [3]:
class Number():
    def __init__(self,number):
        self.number = number

class Variable():
    def __init__(self,symbol):
        self.symbol = symbol

This represents $x^2$:

In [4]:
Power(Variable("x"),Number(2))

In [5]:
class Product():
    def __init__(self, exp1, exp2):
        self.exp1 = exp1
        self.exp2 = exp2

This represents $3x^2$:

In [6]:
Product(Number(3),Power(Variable("x"),Number(2)))

In [7]:
class Sum():
    def __init__(self, *exps): #<1>
        self.exps = exps

class Function(): #<2>
    def __init__(self,name):
        self.name = name

class Apply(): #<3>
    def __init__(self,function,argument):
        self.function = function
        self.argument = argument

f_expression = Product( #<4>
                Sum(
                    Product(
                        Number(3),
                        Power(
                            Variable("x"),
                            Number(2))),
                    Variable("x")),
                Apply(
                    Function("sin"),
                    Variable("x")))

This represents $\cos(x^3 + -5)$:

In [8]:
Apply(Function("cos"),Sum(Power(Variable("x"),Number("3")), Number(-5)))

### Ex10.2
前の練習問題の式をPythonのコードに変換せよ。これをPythonの関数として書き、また、要素とコンビネータを用いたデータ構造にせよ。

In [9]:
from math import log

def ln(y,z):
    return log(y ** z)

ln_expression = Apply(
                    Function("ln"),
                    Power(
                        Variable("y"),
                        Variable("z")
                    ))

### Ex10.4
ある式を別の式で割ったものを表す Quotient コンビネータを実装せよ。次の式はどのように表されるか？
(a + b) / 2

In [10]:
class Quotient():
    def __init__(self, numerator, denominator):
        self.numerator = numerator
        self.denominator = denominator

# (a + b) / 2
Quotient(Sum(Variable("a"), Variable("b")), Number(2))

### Ex10.5
ある式から別の式を引いたものを表す Difference コンビネータを実装せよ。
b^2 - 4ac はどのように表せるか？

In [11]:
class Difference():
    def __init__(self, exp1, exp2):
        self.exp1 = exp1
        self.exp2 = exp2

# b^2 - 4ac
Difference(Power(Variable("b"), Number(2)), Product(Product(Number(4), Variable("a")), Variable("c")))

### Ex10.6
式の符号反転を表す Negative コンビネータを実装せよ。例えば、x^2 + y の反転は -(x^2 + y)となる。
この新しいコンビネータを使って、後者の式をコードで表せ。

In [12]:
class Negative():
    def __init__(self, exp):
        self.exp = exp

Negative(Sum(Power(Variable("x"), Number(2)), Variable("y")))

### Ex10.7
平方根を表す sqrt という関数を追加し、それを使って次の式をコード化せよ。

In [13]:
from math import sqrt as math_sqrt

def sqrt(x):
    return math_sqrt(x)

Quotient(
    Sum(
        Negative(Variable("b")),
        Apply(
                Function("sqrt"),
                Difference(
                    Power(Variable("b"), Number(2)),
                    Product(Product(Number(4), Variable("a")), Variable("c")))
        ),
    ),
    Product(Number(2), Variable("a"))
)

### Ex10.8
Expression という抽象基底クラスを作成し、すべての要素とコンビネータがそれを継承するようにする。
次に Python の算術演算 +、-、*、/ をオーバーロードし、それらが Expression オブジェクトを生成するようにする。

In [14]:
from abc import ABC

class Expression(ABC):
    def __add__(self, other):
        return Sum(self, other)

    def __sub__(self, other):
        return Difference(self, other)

    def __mul__(self, other):
        return Product(self, other)

    def __rmul__(self, other):
        return Product(other, self)

    def __truediv__(self, other):
        return Quotient(self, other)

class Power(Expression):
    def __init__(self,base,exponent):
        self.base = base
        self.exponent = exponent

class Number(Expression):
    def __init__(self,number):
        self.number = number

class Variable(Expression):
    def __init__(self,symbol):
        self.symbol = symbol


class Product(Expression):
    def __init__(self, exp1, exp2):
        self.exp1 = exp1
        self.exp2 = exp2

class Sum(Expression):
    def __init__(self, *exps):
        self.exps = exps

class Function():
    def __init__(self,name):
        self.name = name

class Apply(Expression):
    def __init__(self,function,argument):
        self.function = function
        self.argument = argument

class Quotient(Expression):
    def __init__(self, numerator, denominator):
        self.numerator = numerator
        self.denominator = denominator

class Difference(Expression):
    def __init__(self, exp1, exp2):
        self.exp1 = exp1
        self.exp2 = exp2

class Negative(Expression):
    def __init__(self, exp):
        self.exp = exp

# 2 Putting a symbolic expression to work

## 2.1 Finding all the variables in an expression

In [15]:
def distinct_variables(exp):
    if isinstance(exp, Variable):
        return set(exp.symbol)
    elif isinstance(exp, Number):
        return set()
    elif isinstance(exp, Sum):
        return set().union(*[distinct_variables(exp) for exp in exp.exps])
    elif isinstance(exp, Product):
        return distinct_variables(exp.exp1).union(distinct_variables(exp.exp2))
    elif isinstance(exp, Power):
        return distinct_variables(exp.base).union(distinct_variables(exp.exponent))
    elif isinstance(exp, Apply):
        return distinct_variables(exp.argument)
    else:
        raise TypeError("Not a valid expression.")

In [16]:
distinct_variables(Variable("z"))

{'z'}

In [17]:
distinct_variables(Number(3))

set()

In [18]:
f_expression = Product( #<4>
                Sum(
                    Product(
                        Number(3),
                        Power(
                            Variable("x"),
                            Number(2))),
                    Variable("x")),
                Apply(
                    Function("sin"),
                    Variable("x")))

distinct_variables(f_expression)

{'x'}

## 2.2 Evaluating an expression

In [19]:
from abc import ABC, abstractmethod

class Expression(ABC):
    @abstractmethod
    def evaluate(self, **bindings):
        pass

Note: we are redefining these classes now.

In [20]:
class Number(Expression):
    def __init__(self,number):
        self.number = number
    def evaluate(self, **bindings):
        return self.number

class Variable(Expression):
    def __init__(self,symbol):
        self.symbol = symbol
    def evaluate(self, **bindings):
        try:
            return bindings[self.symbol]
        except:
            raise KeyError("Variable '{}' is not bound.".format(self.symbol))

class Product(Expression):
    def __init__(self, exp1, exp2):
        self.exp1 = exp1
        self.exp2 = exp2
    def evaluate(self, **bindings):
        return self.exp1.evaluate(**bindings) * self.exp2.evaluate(**bindings)

In [21]:
Product(Variable("x"), Variable("y")).evaluate(x=2,y=5)

10

In [22]:
import math
from math import sin, cos, log

_function_bindings = {
    "sin": math.sin,
    "cos": math.cos,
    "ln": math.log
}

class Apply(Expression):
    def __init__(self,function,argument):
        self.function = function
        self.argument = argument
    def evaluate(self, **bindings):
        return _function_bindings[self.function.name](self.argument.evaluate(**bindings))

From the text: *... Similarly, we can add an “evaluate” method to the Sum, Power, Difference, or Quotient combinators....*

In [23]:
class Sum(Expression):
    def __init__(self, *exps):
        self.exps = exps
    def evaluate(self, **bindings):
        return sum([exp.evaluate(**bindings) for exp in self.exps])

class Power(Expression):
    def __init__(self,base,exponent):
        self.base = base
        self.exponent = exponent
    def evaluate(self, **bindings):
        return self.base.evaluate(**bindings) ** self.exponent.evaluate(**bindings)

class Difference(Expression):
    def __init__(self,exp1,exp2):
        self.exp1 = exp1
        self.exp2 = exp2
    def evaluate(self, **bindings):
        return self.exp1.evaluate(**bindings) - self.exp2.evaluate(**bindings)

class Quotient(Expression):
    def __init__(self,numerator,denominator):
        self.numerator = numerator
        self.denominator = denominator
    def evaluate(self, **bindings):
        return self.numerator.evaluate(**bindings) / self.denominator.evaluate(**bindings)

Redefine `f_expression` in light of the new class definitions

In [24]:
f_expression = Product( #<4>
                Sum(
                    Product(
                        Number(3),
                        Power(
                            Variable("x"),
                            Number(2))),
                    Variable("x")),
                Apply(
                    Function("sin"),
                    Variable("x")))

In [25]:
f_expression.evaluate(x=5)

-76.71394197305108

In [26]:
from math import sin
def f(x):
    return (3*x**2 + x) * sin(x)

f(5)

-76.71394197305108

## 2.3 Expanding an expression

In [27]:
class Expression(ABC):
    @abstractmethod
    def evaluate(self, **bindings):
        pass
    @abstractmethod
    def expand(self):
        pass

    # Printing expressions legibly in REPL (See first mini project in 2.4)
    @abstractmethod
    def display(self):
        pass
    def __repr__(self):
        return self.display()

In [28]:
class Sum(Expression):
    def __init__(self, *exps):
        self.exps = exps
    def evaluate(self, **bindings):
        return sum([exp.evaluate(**bindings) for exp in self.exps])
    def expand(self):
        return Sum(*[exp.expand() for exp in self.exps])
    def display(self):
        return "Sum({})".format(",".join([e.display() for e in self.exps]))

class Product(Expression):
    def __init__(self, exp1, exp2):
        self.exp1 = exp1
        self.exp2 = exp2
    def evaluate(self, **bindings):
        return self.exp1.evaluate(**bindings) * self.exp2.evaluate(**bindings)
    def expand(self):
        expanded1 = self.exp1.expand()
        expanded2 = self.exp2.expand()
        if isinstance(expanded1, Sum):
            return Sum(*[Product(e,expanded2).expand() for e in expanded1.exps])
        elif isinstance(expanded2, Sum):
            return Sum(*[Product(expanded1,e) for e in expanded2.exps])
        else:
            return Product(expanded1,expanded2)
    def display(self):
        return "Product({},{})".format(self.exp1.display(),self.exp2.display())

class Difference(Expression):
    def __init__(self,exp1,exp2):
        self.exp1 = exp1
        self.exp2 = exp2
    def evaluate(self, **bindings):
        return self.exp1.evaluate(**bindings) - self.exp2.evaluate(**bindings)
    def expand(self):
        return self
    def display(self):
        return "Difference({},{})".format(self.exp1.display(), self.exp2.display())

class Quotient(Expression):
    def __init__(self,numerator,denominator):
        self.numerator = numerator
        self.denominator = denominator
    def evaluate(self, **bindings):
        return self.numerator.evaluate(**bindings) / self.denominator.evaluate(**bindings)
    def expand(self):
        return self
    def display(self):
        return "Quotient({},{})".format(self.numerator.display(),self.denominator.display())

class Negative(Expression):
    def __init__(self,exp):
        self.exp = exp
    def evaluate(self, **bindings):
        return - self.exp.evaluate(**bindings)
    def expand(self):
        return self
    def display(self):
        return "Negative({})".format(self.exp.display())

class Number(Expression):
    def __init__(self,number):
        self.number = number
    def evaluate(self, **bindings):
        return self.number
    def expand(self):
        return self
    def display(self):
        return "Number({})".format(self.number)

class Power(Expression):
    def __init__(self,base,exponent):
        self.base = base
        self.exponent = exponent
    def evaluate(self, **bindings):
        return self.base.evaluate(**bindings) ** self.exponent.evaluate(**bindings)
    def expand(self):
        return self
    def display(self):
        return "Power({},{})".format(self.base.display(),self.exponent.display())

class Variable(Expression):
    def __init__(self,symbol):
        self.symbol = symbol
    def evaluate(self, **bindings):
        return bindings[self.symbol]
    def expand(self):
        return self
    def display(self):
        return "Variable(\"{}\")".format(self.symbol)

class Function():
    def __init__(self,name,make_latex=None):
        self.name = name
        self.make_latex = make_latex
    def latex(self,arg_latex):
        if self.make_latex:
            return self.make_latex(arg_latex)
        else:
            return " \\operatorname{{ {} }} \\left( {} \\right)".format(self.name, arg_latex)

class Apply(Expression):
    def __init__(self,function,argument):
        self.function = function
        self.argument = argument
    def evaluate(self, **bindings):
        return _function_bindings[self.function.name](self.argument.evaluate(**bindings))
    def expand(self):
        return Apply(self.function, self.argument.expand())
    def display(self):
        return "Apply(Function(\"{}\"),{})".format(self.function.name, self.argument.display())

In [29]:
Y = Variable('y')
Z = Variable('z')
A = Variable('a')
B = Variable('b')
Product(Sum(A,B),Sum(Y,Z))

Product(Sum(Variable("a"),Variable("b")),Sum(Variable("y"),Variable("z")))

In [30]:
Product(Sum(A,B),Sum(Y,Z)).expand()

Sum(Sum(Product(Variable("a"),Variable("y")),Product(Variable("a"),Variable("z"))),Sum(Product(Variable("b"),Variable("y")),Product(Variable("b"),Variable("z"))))

In [31]:
f_expression = Product( #<4>
                Sum(
                    Product(
                        Number(3),
                        Power(
                            Variable("x"),
                            Number(2))),
                    Variable("x")),
                Apply(
                    Function("sin"),
                    Variable("x")))

In [32]:
f_expression.expand()

Sum(Product(Product(Number(3),Power(Variable("x"),Number(2))),Apply(Function("sin"),Variable("x"))),Product(Variable("x"),Apply(Function("sin"),Variable("x"))))

# 3 Finding the derivative of a function

For the rest of the notebook, I'll use the complete implementations from `expressions.py` so I don't have to re-implement every time.

In [33]:
from expressions import *

In [34]:
Product(Power(Variable("x"),Number(2)),Apply(Function("sin"),Variable("y")))

Product(Power(Variable("x"),Number(2)),Apply(Function("sin"),Variable("y")))

## 3.1 Derivatives of powers

## 3.2 Derivatives of transformed functions

## 3.3 Derivatives of some special functions

## 3.4 Derivatives of products and compositions


## 3.5 Exercises

# 4 Taking derivatives automatically

## 4.1 Implementing a derivative method for expressions

In [35]:
Sum(Variable("x"),Variable("c"),Number(1)).derivative(Variable("x"))

Sum(Number(1),Number(0),Number(0))

## 4.2 Implementing the product rule and chain rule

In [36]:
Product(Variable("c"),Variable("x")).derivative(Variable("x"))

Product(Variable("c"),Number(1))

In [37]:
Apply(Function("sin"),Power(Variable("x"),Number(2))).derivative(x)

Product(Product(Number(1),Product(Number(2),Power(Variable("x"),Number(1)))),Apply(Function("cos"),Power(Variable("x"),Number(2))))

## 4.3 Implementing the power rule

In [38]:
f_expression = Product( #<4>
                Sum(
                    Product(
                        Number(3),
                        Power(
                            Variable("x"),
                            Number(2))),
                    Variable("x")),
                Apply(
                    Function("sin"),
                    Variable("x")))

In [40]:
f_expression.derivative(x)

Sum(Product(Sum(Product(Number(3),Product(Number(1),Product(Number(2),Power(Variable("x"),Number(1))))),Number(1)),Apply(Function("sin"),Variable("x"))),Product(Sum(Product(Number(3),Power(Variable("x"),Number(2))),Variable("x")),Product(Number(1),Apply(Function("cos"),Variable("x")))))

## 4.4 Exercises

# 5 Integrating functions symbolically 

## 5.1 Integrals as antiderivatives


## 5.2 Introducing the SymPy library

In [41]:
from sympy import *
from sympy.core.core import *
Mul(Symbol('y'),Add(3,Symbol('x')))

y*(x + 3)

In [42]:
y = Symbol('y')

In [43]:
x = Symbol('x')

In [44]:
y*(3+x)

y*(x + 3)

In [45]:
y*(3+x).subs(x,1)

4*y

In [46]:
(x**2).diff(x)

2*x

In [47]:
(3*x**2).integrate(x)

x**3